In [4]:
import torch
import torchaudio
from datasets import load_dataset, load_metric
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import re
import unidecode

/opt/conda/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [5]:
lang_code = "rw"
language = "kinyarwanda"
model = f"lucio/wav2vec2-large-xlsr-{language}"

In [6]:
import numpy as np

np.array(3)


array(3)

In [125]:

processor = Wav2Vec2Processor.from_pretrained(model)
model = Wav2Vec2ForCTC.from_pretrained(model)
model.to("cuda")

wer = load_metric("wer")


Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureExtractor(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (1): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (2): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (3): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (4): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,)

In [18]:
## Text pre-processing

chars_to_ignore_regex = r'[\[\],?.!;:*½¼%+=$$€™&©°"‘’“”(){}‟ˮ‘’´`ʺ″«»<>/…‽�–]'
chars_to_ignore_pattern = re.compile(chars_to_ignore_regex)

def remove_special_characters(batch):
    batch["text"] = re.sub(r'(\w)[‘’´`](\w)', r"\1'\2", batch["sentence"])
    batch["text"] = re.sub(chars_to_ignore_regex, "", batch["text"]).lower().strip() + " "
    batch["text"] = re.sub(r"(-|' | '|  +)", " ", batch["text"])
    batch["text"] = unidecode.unidecode(batch["text"])
    batch["length"] = len(batch["text"])
    return batch


## Audio pre-processing
resampler = torchaudio.transforms.Resample(48_000, 16_000)

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    batch["sampling_rate"] = 16_000
    return batch


# Text transformation and audio resampling
def cv_prepare(batch):
    batch = remove_special_characters(batch)
    batch = speech_file_to_array_fn(batch)

    return batch

# Number of CPUs or None
num_proc = 4



In [129]:
test_dataset = load_dataset("common_voice", lang_code, split="test", cache_dir=f"/workspace/raw_data/{lang_code}")

test_dataset = test_dataset.map(cv_prepare, num_proc=num_proc)

Reusing dataset common_voice (/workspace/raw_data/lg/common_voice/lg/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f)


In [130]:
def evaluate(batch):
    inputs = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

    with torch.no_grad():
        logits = model(inputs.input_values.to("cuda"), attention_mask=inputs.attention_mask.to("cuda")).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_strings"] = processor.batch_decode(pred_ids)
    return batch

result = test_dataset.map(evaluate, batched=True, batch_size=8)


RuntimeError: CUDA out of memory. Tried to allocate 454.00 MiB (GPU 0; 31.75 GiB total capacity; 1.66 GiB already allocated; 198.50 MiB free; 1.72 GiB reserved in total by PyTorch)

In [24]:
Counter(zip(test_dataset['age'], test_dataset['gender']))

Counter({('teens', 'male'): 2251,
         ('twenties', 'female'): 6235,
         ('teens', 'female'): 1755,
         ('twenties', 'male'): 3451,
         ('twenties', ''): 1340})

In [7]:
# WER Metric computation

import jiwer

def chunked_wer(targets, predictions, chunk_size=None):                                          
    if chunk_size is None: return jiwer.wer(targets, predictions)                                
    start = 0                                                                                    
    end = chunk_size                                                                             
    H, S, D, I = 0, 0, 0, 0                                                                      
    while start < len(targets):                                                                  
        chunk_metrics = jiwer.compute_measures(targets[start:end], predictions[start:end])       
        H = H + chunk_metrics["hits"]                                                            
        S = S + chunk_metrics["substitutions"]                                                   
        D = D + chunk_metrics["deletions"]                                                       
        I = I + chunk_metrics["insertions"]                                                      
        start += chunk_size                                                                      
        end += chunk_size                                                                        
    return float(S + D + I) / float(H + S + D)

print("WER: {:2f}".format(100 * chunked_wer(result["sentence"], result["pred_strings"], chunk_size=4000)))


WER: 29.805760


In [119]:


def extract_all_chars(batch):
    all_text = " ".join(batch["text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

def strip_accents(batch):
   batch["sentence"] = unidecode.unidecode(batch["sentence"])


In [7]:
valid_dataset = load_dataset("common_voice", "rw", split="validation", cache_dir="/workspace/raw_data/rw")
valid_dataset = valid_dataset.map(remove_special_characters, remove_columns=['path'], num_proc=4)


Reusing dataset common_voice (/workspace/raw_data/rw/common_voice/rw/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f)


In [9]:
len(valid_dataset), valid_dataset[0]

(15032,
 {'accent': '',
  'age': 'teens',
  'client_id': '5e85947436bbc805e503bf926beb533b07da07cef77c491a476c8cd1d9bce7b9a4cdc8171564447ea1d2c9c591a203b2e37545a4c8a8f7d0b2881e9e64441e5b',
  'down_votes': 1,
  'gender': 'male',
  'locale': 'rw',
  'segment': "''",
  'sentence': 'Umuntungo we ntabwo aba agomba kuwumarira mu gushakisha imyenda yo kwambara.',
  'text': 'umuntungo we ntabwo aba agomba kuwumarira mu gushakisha imyenda yo kwambara ',
  'up_votes': 2})

In [19]:
train_dataset = load_dataset("common_voice", "rw", split="train[:2%]", cache_dir="/workspace/raw_data/rw")
train_dataset = train_dataset.map(remove_special_characters, remove_columns=['path'], num_proc=4)

Reusing dataset common_voice (/workspace/raw_data/rw/common_voice/rw/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f)


In [20]:
len(train_dataset), train_dataset[0], train_dataset["length"]

(10304,
 {'accent': '',
  'age': 'twenties',
  'client_id': 'd4439c64c8f13b84cd2ce31d5d9eeae2a81147d89abb00cebaaf11b60b7166c24dd257a44e73c72c73c93cae29d904bed135824aca06e5970e001e9406e8a891',
  'down_votes': 1,
  'gender': 'male',
  'length': 53,
  'locale': 'rw',
  'segment': "''",
  'sentence': 'akunda u rwanda cyane cyane ku byerekeye isuku ihaba',
  'text': 'akunda u rwanda cyane cyane ku byerekeye isuku ihaba ',
  'up_votes': 2},
 [53,
  58,
  44,
  37,
  164,
  38,
  48,
  36,
  37,
  49,
  44,
  41,
  21,
  47,
  24,
  54,
  28,
  78,
  45,
  24,
  42,
  48,
  36,
  47,
  56,
  54,
  71,
  24,
  27,
  28,
  51,
  34,
  39,
  63,
  46,
  29,
  83,
  38,
  47,
  84,
  32,
  85,
  41,
  38,
  25,
  38,
  37,
  62,
  94,
  56,
  34,
  37,
  103,
  66,
  33,
  47,
  79,
  39,
  30,
  26,
  44,
  80,
  35,
  49,
  36,
  50,
  40,
  46,
  42,
  32,
  64,
  48,
  76,
  49,
  41,
  40,
  36,
  57,
  37,
  38,
  32,
  26,
  19,
  46,
  29,
  46,
  33,
  42,
  51,
  36,
  42,
  44,
  42,
 

In [13]:
from collections import Counter 

Counter(zip(valid_dataset['age'], valid_dataset['gender']))

Counter({('teens', 'male'): 2251,
         ('twenties', 'female'): 6235,
         ('teens', 'female'): 1755,
         ('twenties', 'male'): 3451,
         ('twenties', ''): 1340})

In [14]:
Counter(zip(train_dataset['age'], train_dataset['gender']))

Counter({('twenties', 'male'): 16411,
         ('teens', 'female'): 12038,
         ('thirties', 'male'): 5512,
         ('twenties', ''): 5628,
         ('thirties', 'female'): 2134,
         ('twenties', 'female'): 6273,
         ('teens', 'male'): 3524})

In [121]:
vocab = valid_dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=valid_dataset.column_names,
)


In [19]:
vocab_test = test_dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=test_dataset.column_names,
)

NameError: name 'test_dataset' is not defined

In [122]:
from collections import Counter
cnts = Counter(vocab[0]['all_text'])
cnts

Counter({'u': 66225,
         'm': 36247,
         'n': 52029,
         't': 22892,
         'g': 29882,
         'o': 40150,
         ' ': 146733,
         'w': 22352,
         'e': 52965,
         'a': 134320,
         'b': 46183,
         'k': 36256,
         'r': 47648,
         'i': 84230,
         's': 19735,
         'h': 20429,
         'y': 35120,
         'd': 11235,
         'c': 6360,
         'l': 3378,
         "'": 7024,
         'z': 17673,
         'j': 4789,
         'f': 4162,
         'v': 3791,
         'p': 3312,
         'x': 67,
         'q': 72})

In [82]:
import unidecode
[(unidecode.unidecode(k), cnts[k]) for k in cnts if cnts[k] < 100]

[('x', 67), ('q', 72), ('<', 60), ('>', 30), ('$', 3), ('4', 2), ('+', 1)]

In [123]:
list(zip(valid_dataset['sentence'][:100], valid_dataset['text'][:100]))

[('Umuntungo we ntabwo aba agomba kuwumarira mu gushakisha imyenda yo kwambara.',
  'umuntungo we ntabwo aba agomba kuwumarira mu gushakisha imyenda yo kwambara '),
 ('Richard Gordon Turnbull ko atemeranya n’abashobora gukandamiza abandi bitwaje ibara ry’uruhu.',
  "richard gordon turnbull ko atemeranya n'abashobora gukandamiza abandi bitwaje ibara ry'uruhu "),
 ('Ni inka ikurikiranwa hifashishijwe ikoranabuhanga, buri uko yabira biba bisuzumwa.',
  'ni inka ikurikiranwa hifashishijwe ikoranabuhanga buri uko yabira biba bisuzumwa '),
 ('Ati, “Benshi bafatwa baba ari urubyiruko, hari ababa banyoye ibiyobyabwenge.',
  'ati benshi bafatwa baba ari urubyiruko hari ababa banyoye ibiyobyabwenge '),
 ('Olivis : Inzozi ze ni uguhura na Producer ukomeye muri Amerika “Timberland” akabakorera indirimbo.',
  'olivis inzozi ze ni uguhura na producer ukomeye muri amerika timberland akabakorera indirimbo '),
 ('ibitaramo bitandukanye mu Rwanda avuga ko yari abikumbuye ariko ngo byamubereye.',
  'ibit

In [47]:
import unicodedata
unicodedata.name('́')

'COMBINING ACUTE TONE MARK'

In [67]:
processor.tokenizer.get_vocab()

{'y': 0,
 'd': 1,
 's': 2,
 'n': 3,
 'c': 4,
 'l': 5,
 'h': 6,
 'w': 7,
 'u': 8,
 'z': 9,
 't': 11,
 'g': 12,
 'j': 13,
 'm': 14,
 'o': 15,
 'e': 16,
 'k': 17,
 'r': 18,
 'i': 19,
 'b': 20,
 'f': 21,
 'a': 22,
 'v': 23,
 'p': 24,
 '|': 10,
 '[UNK]': 25,
 '[PAD]': 26,
 '<s>': 27,
 '</s>': 28}

In [77]:
from transformers.trainer_utils import get_last_checkpoint
import os
import logging

logger = logging.getLogger(__name__)

In [71]:
get_last_checkpoint("/workspace/checkpoints/lg/wav2vec2-large-xlsr-lg-augment")

'/workspace/checkpoints/lg/wav2vec2-large-xlsr-lg-augment/checkpoint-4096'

In [75]:
output_dir="/workspace/checkpoints/lg/wav2vec2-large-xlsr-lg-augment/" 

In [82]:
last_checkpoint = None
if os.path.isdir(output_dir) and True and not False:
    print(f"Looking for last checkpoint in {output_dir}")
    last_checkpoint = get_last_checkpoint(output_dir)
    if last_checkpoint is None and len(os.listdir(output_dir)) > 0:
        raise ValueError(
            f"Output directory ({output_dir}) already exists and is not empty. "
            "Use --overwrite_output_dir to overcome."
        )
    elif last_checkpoint is not None:
        print(
            f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
            "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
        )

Looking for last checkpoint in /workspace/checkpoints/lg/wav2vec2-large-xlsr-lg-augment/
Checkpoint detected, resuming training at /workspace/checkpoints/lg/wav2vec2-large-xlsr-lg-augment/checkpoint-4096. To avoid this behavior, change the `--output_dir` or add `--overwrite_output_dir` to train from scratch.


In [81]:
os.path.isdir(output_dir) and True and not False

True

In [152]:
import librosa
import torchaudio
import soundfile
from audiomentations import Compose, AddGaussianNoise, Gain, PitchShift, Shift

resampler = torchaudio.transforms.Resample(48_000, 16_000)

augment = Compose([
    AddGaussianNoise(min_amplitude=0.0001, max_amplitude=0.01, p=1),
    PitchShift(min_semitones=-3, max_semitones=3, p=0.8),
    Gain(min_gain_in_db=-6, max_gain_in_db=6, p=0.8),
    Shift(min_fraction=-0.5, max_fraction=0.5, p=0.8),
])

def speech_file_to_array_fn_torchaudio(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    batch["sampling_rate"] = 16_000
    return batch

def speech_file_to_array_fn_librosa(batch):
    speech_array, sample_rate = torchaudio.load(batch["path"])
    batch["speech"] = librosa.resample(speech_array.squeeze().numpy(), sample_rate, 16_000)
    return batch

def augmented_speech_file_to_array_fn(batch):
    try:
        speech_array, sampling_rate = soundfile.read(batch["path"] + "-augmented.wav")
    except:
        speech_array, sampling_rate = torchaudio.load(batch["path"])
        speech_array = resampler(speech_array)
        speech_array = augment(samples=speech_array, sample_rate=sampling_rate).squeeze()
        soundfile.write(batch["path"]+"-augmented.wav", speech_array, sampling_rate, subtype='PCM_24')

    batch["speech"] = speech_array
    batch["sampling_rate"] = 16_000
    return batch

In [153]:
speech_file_to_array_fn_librosa(test_dataset[23])

{'accent': '',
 'age': '',
 'client_id': 'a0c75a2b3ef19a055ebcc7587ebf341614373519ceb63f67e67e4eae0eb4858dc2031cd42b08b389168e6c84bce198dc8f7405fdb443195eaaf6cd7dc981425c',
 'down_votes': 1,
 'gender': '',
 'locale': 'rw',
 'path': '/workspace/raw_data/rw/downloads/extracted/6755b117c96a17977cf94d8a118d06e13daa8d123092dfaa6ccb5bd13f06e60f/cv-corpus-6.1-2020-12-11/rw/clips/common_voice_rw_22948194.mp3',
 'segment': "''",
 'sentence': 'ni uko batari bagiha agaciro isanduku n’ibyo yavuze ku ri bo',
 'up_votes': 2,
 'speech': array([ 0.        ,  0.        ,  0.        , ..., -0.00116267,
        -0.00045365, -0.00068105], dtype=float32)}

In [154]:
augmented_speech_file_to_array_fn(test_dataset[24])

{'accent': '',
 'age': '',
 'client_id': 'a0c75a2b3ef19a055ebcc7587ebf341614373519ceb63f67e67e4eae0eb4858dc2031cd42b08b389168e6c84bce198dc8f7405fdb443195eaaf6cd7dc981425c',
 'down_votes': 1,
 'gender': '',
 'locale': 'rw',
 'path': '/workspace/raw_data/rw/downloads/extracted/6755b117c96a17977cf94d8a118d06e13daa8d123092dfaa6ccb5bd13f06e60f/cv-corpus-6.1-2020-12-11/rw/clips/common_voice_rw_22948195.mp3',
 'segment': "''",
 'sentence': 'amata kwera zari zikeye mu maso kurusha',
 'up_votes': 2,
 'speech': array([-0.43122205, -0.42071682, -0.4279239 , ..., -0.4282133 ,
        -0.4141648 , -0.3871291 ], dtype=float32),
 'sampling_rate': 16000}

In [155]:
augmented_speech_file_to_array_fn(test_dataset[24])['speech'].squeeze()

array([-0.05630261, -0.03208497, -0.05551476, ...,  0.00743804,
       -0.00030689, -0.08274259], dtype=float32)

In [156]:
test_dataset.map(augmented_speech_file_to_array_fn)

KeyboardInterrupt: 